# Sensitivity Studies

Compare with:

+ Arc
+ Act 1 & 2
+ Demo (Pulsed and SS)

// also adds prototype reactor designs (scylla and charybdis)

## 0.0 Bootup Notebook

In [16]:
if Sys.CPU_CORES != nprocs()
    rmprocs(collect(2:100))
    addprocs(Sys.CPU_CORES-1)
end

@everywhere using Fussy
Main.IJulia.set_current_module(Fussy)

return

## 1.0 Set Defaults

In [17]:
cur_decks = [
    :scylla, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed
]
  
cur_params = [ 
    :wave_theta, :H, :Q, 
    :epsilon, :kappa_95, :delta_95,
    :nu_n, :nu_T, :l_i, 
    :N_G, :f_D, :Z_eff,
    :max_beta_N, :max_q_95, :max_P_W, 
    :eta_CD, :B_CS, :tau_FT, 
    :A, :C_ejima
]

return

## 2.0 Define Make Function

In [18]:
function make_studies(cur_param)
    tmp_studies = Dict()

    for cur_deck in cur_decks
        println(cur_deck)
        
        if cur_deck == :demo_pulsed || cur_deck == :scylla || cur_deck == :demo_steady
            is_consistent = false
        else
            is_consistent = true
        end
        
        cur_study = Study(cur_param, deck=cur_deck, is_consistent=is_consistent, sensitivity=1.0, num_points=41, is_fast=true)
        tmp_studies[cur_deck] = cur_study
        sleep(0.25)
    end

    println("done.")

    tmp_studies
end

return

## 3.0 Get Data

In [4]:
using FileIO
using JLD2

In [10]:
has_studies = isdefined(Fussy, :cur_studies)

isdefined(Fussy, :cur_studies) || ( cur_studies = Dict() )

cur_file = "../data/sensitivities.jld2"

if !has_studies && isfile(cur_file)
    has_studies = true
    try
        cur_studies = load(cur_file, "cur_studies")
    catch
        has_studies = false
    end
end

latent_error = nothing
for cur_param in cur_params
    haskey(cur_studies, cur_param) && continue
    println("> ", cur_param, "\n")
    try
        cur_studies[cur_param] = make_studies(cur_param)
    catch cur_error
        latent_error = cur_error
        break
    end
end

isempty(cur_studies) || save(cur_file, Dict("cur_studies" => cur_studies))

( latent_error == nothing ) || rethrow(latent_error)

return

> C_ejima

scylla


Progress: 100%|█████████████████████████████████████████| Time: 0:00:22
Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


charybdis


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:33


arc


...

demo_steady


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


demo_pulsed


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


done.


## 4.0 Setup for Plots

In [11]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d"
]

sort!(cur_fields)

unshift!(cur_fields, "R_0")

return

In [12]:
using Plots

# plotlyjs()

In [13]:
using Interact
using StringCases
using DataStructures

## 5.0 Make Plot GUI

In [19]:
@manipulate for xvar in cur_params, yvar in cur_fields, deck in cur_decks
    cur_study = cur_studies[xvar][deck]

    work_dict = OrderedDict()
    
    work_dict["Wall"] = cur_study.wall_reactors
    work_dict["Kink"] = cur_study.kink_reactors
    work_dict["Cost"] = cur_study.cost_reactors
    
    filter!(work_reac -> work_reac.is_valid, work_dict["Wall"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Kink"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Cost"])

    max_yy = 0
    
    plot()
    for (cur_index, (cur_key, cur_value)) in enumerate(work_dict)
        isempty(cur_value) && continue
        cur_xx = map(cur_reactor -> getfield(cur_reactor, Symbol(xvar)), cur_value) 
        cur_yy = map(cur_reactor -> getfield(cur_reactor, Symbol(yvar)), cur_value) 

        max_yy = max(max_yy, maximum(cur_yy))
        
        cur_label = ( cur_key == "Cost" ) ? "Cost" : "Beta-$(cur_key)"
        plot!(cur_xx, cur_yy, label=cur_label, color=cur_index)
    end
    plot!()
    
    cur_bot_value = (1-cur_study.sensitivity)*cur_study.default
    cur_top_value = (1+cur_study.sensitivity)*cur_study.default
    
    plot!(
        [cur_study.default,cur_study.default],
        [0,max_yy*1.25], 
        label="Default", style=:dot, color=4, lineopacity=0.55, width=1.5
    )
    
    plot!(
        [cur_bot_value,cur_bot_value,NaN,cur_top_value,cur_top_value],
        [0,max_yy*1.25,NaN,0,max_yy*1.25], 
        label="Edge", style=:dot, color=5, lineopacity=0.55, width=1.5
    )
    
    max_xx = cur_top_value
    
    xlims!(0, max_xx * 1.25)
    ylims!(0, max_yy * 1.25)

    xlabel!(string(xvar))
    ylabel!(yvar)

    cur_title = join(map(capitalize, split(string(deck), "_")), " ")
    title!(cur_title)
end

Interact.Options{:ToggleButtons,Symbol}(17: "input-7" = wave_theta Symbol , "xvar", :wave_theta, "wave_theta", 1, Interact.OptionDict(DataStructures.OrderedDict("wave_theta"=>:wave_theta,"H"=>:H,"Q"=>:Q,"epsilon"=>:epsilon,"kappa_95"=>:kappa_95,"delta_95"=>:delta_95,"nu_n"=>:nu_n,"nu_T"=>:nu_T,"l_i"=>:l_i,"N_G"=>:N_G…), Dict(:tau_FT=>"tau_FT",:f_D=>"f_D",:max_beta_N=>"max_beta_N",:max_q_95=>"max_q_95",:kappa_95=>"kappa_95",:eta_CD=>"eta_CD",:epsilon=>"epsilon",:nu_n=>"nu_n",:C_ejima=>"C_ejima",:A=>"A"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,String}(19: "input-8" = R_0 String , "yvar", "R_0", "R_0", 1, Interact.OptionDict(DataStructures.OrderedDict("R_0"=>"R_0","B_0"=>"B_0","I_P"=>"I_P","P_F"=>"P_F","R_CS"=>"R_CS","T_bar"=>"T_bar","W_M"=>"W_M","b"=>"b","c"=>"c","cost"=>"cost"…), Dict("W_M"=>"W_M","I_P"=>"I_P","c"=>"c","R_0"=>"R_0","B_0"=>"B_0","b"=>"b","n_bar"=>"n_bar","eta_CD"=>"eta_CD","f_IN"=>"f_IN","norm_P_W"=>"norm_P_W"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,Symbol}(21: "input-9" = scylla Symbol , "deck", :scylla, "scylla", 1, Interact.OptionDict(DataStructures.OrderedDict("scylla"=>:scylla,"charybdis"=>:charybdis,"arc"=>:arc,"act_1"=>:act_1,"act_2"=>:act_2,"demo_steady"=>:demo_steady,"demo_pulsed"=>:demo_pulsed), Dict(:act_1=>"act_1",:scylla=>"scylla",:demo_steady=>"demo_steady",:charybdis=>"charybdis",:act_2=>"act_2",:arc=>"arc",:demo_pulsed=>"demo_pulsed")), Any[], Any[], true, "horizontal", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 Scylla 
 
 
 wave_theta 
 
 
 R_0 
 
<polyline clip-path="url(#clip5102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.1866,125.146 49.0292,125.146 59.8718,125.146 70.7143,125.146 81.5569,125.146 92.3995,125.146 103.242,125.146 114.085,125.146 124.927,125.146 135.77,125.146 
 146.612,125.146 157.455,125.146 168.297,125.146 179.14,125.146 189.983,125.146 200.825,125.146 211.668,125.146 222.51,125.146 233.353,125.146 244.195,125.146 
 255.038,125.146 265.881,125.146 276.723,125.146 287.566,125.146 298.408,125.146 309.251,125.146 320.093,125.146 330.936,125.146 341.779,125.146 352.621,125.146 
 363.464,125.146 374.306,125.146 385.149,125.146 395.991,125.146 406.834,125.146 417.676,125.146 428.519,125.146 439.362,125.146 450.204,125.146 461.047,125.146 
 471.889,125.146 
 "/>
<polyline clip-path="url(#clip5102)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.1866,97.221 49.0292,97.221 59.8718,97.221 70.7143,97.221 81.5569,97.221 92.3995,97.221 103.242,97.221 114.085,97.221 124.927,97.221 135.77,97.221 
 146.612,97.221 157.455,97.221 168.297,97.221 179.14,97.221 189.983,97.221 200.825,97.221 211.668,97.221 222.51,97.221 233.353,97.221 244.195,97.221 
 255.038,97.221 265.881,97.221 276.723,97.221 287.566,97.221 298.408,97.221 309.251,97.221 320.093,97.221 330.936,97.221 341.779,97.221 352.621,97.221 
 363.464,97.221 374.306,97.221 385.149,97.221 395.991,97.221 406.834,97.221 417.676,97.221 428.519,97.221 439.362,97.221 450.204,97.221 461.047,97.221 
 471.889,97.221 
 "/>
 
 
 
 
 
 
 
 Beta-Wall 
 
 
 
 Beta-Kink 
 
 
 
 Default 
 
 
 
 Edge